In [1]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
%config InlineBackend.figure_format = 'retina'

In [4]:
from pathlib import Path
import numpy as np
from astropy.io import fits
from astropy.table import Table
from matplotlib import pyplot as plt
from kspecdr.inst.isoplane import convert_isoplane_header, add_fiber_table
from kspecdr.io.image import ImageFile
from kspecdr.preproc.make_im import make_im
from kspecdr.preproc.preproc import reduce_bias, reduce_dark
from kspecdr.tlm.make_tlm import read_instrument_data, make_tlm
from scipy.signal import find_peaks_cwt, find_peaks
from rascal.util import refine_peaks
from matplotlib.lines import Line2D

WD = Path("/data1/hbahk/kspec/kspecdr")
RESOURCES = WD / "resources"
TESTDIR = RESOURCES / "isoplane_commissioning"

/home/hbahk/miniconda3/envs/twodfdr/lib/python3.12/site-packages/rascal/calibrator.py:9: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [13]:
bias_files = list(TESTDIR.glob("bias_*.fits"))
# dark_files = list(TESTDIR.glob("dark_*.fits"))

bias_files_converted = [TESTDIR / f.name.replace(".fits", "c.fits") for f in bias_files]

for bfpath, bfpath_converted in zip(bias_files, bias_files_converted):
    hdul = fits.open(bfpath)
    hdr = hdul[0].header
    new_hdr = convert_isoplane_header(hdr, ndfclass="BIAS")
    hdul[0].header = new_hdr
    # use the first frame - (1, 1340, 1300) --> (1340, 1300)
    hdul[0].data = hdul[0].data[0]
    # make new fits file with new header and fiber table
    new_hdr["NAXIS"] = 2
    # new_hdr.remove("NAXIS3")
    hdul[0].header = new_hdr
    hdul.writeto(bfpath_converted, overwrite=True)


mbias_path = TESTDIR / "mbias.fits"
mbias_file = reduce_bias(bias_files_converted, output_file=mbias_path.as_posix())

# mdark_path = TESTDIR / "mdark.fits"
# mdark_file = reduce_dark(
#     dark_files, output_file=mdark_path.as_posix(), bias_filename=mbias_path.as_posix()
# )

INFO:kspecdr.preproc.make_im:==================================================
INFO:kspecdr.preproc.make_im:Preprocessing image data contained in RAW frame
INFO:kspecdr.preproc.make_im:==================================================
INFO:kspecdr.preproc.make_im:RAW file = /data1/hbahk/kspec/kspecdr/resources/isoplane_commissioning/bias_00001c.fits
INFO:kspecdr.preproc.make_im:Creating IM file from raw data with TDFIO_CREATEBYCOPY functionality...
INFO:kspecdr.preproc.make_im:Creating IM file from raw file: /data1/hbahk/kspec/kspecdr/resources/isoplane_commissioning/bias_00001c.fits -> /data1/hbahk/kspec/kspecdr/resources/isoplane_commissioning/bias_00001c_im.fits
INFO:kspecdr.io.image:Opened file: /data1/hbahk/kspec/kspecdr/resources/isoplane_commissioning/bias_00001c.fits (mode: READ)
INFO:kspecdr.preproc.make_im:Instrument: ISOPLANE, SPECTID: UNKNOWN, CLASS: , BITPIX: 16
INFO:kspecdr.io.image:Closed file: /data1/hbahk/kspec/kspecdr/resources/isoplane_commissioning/bias_00001c.fit

ValueError: could not broadcast input array from shape (1340,1300) into shape (1300,1340)

In [6]:
bias_files

[PosixPath('/data1/hbahk/kspec/kspecdr/resources/isoplane_commissioning/bias_00001.fits'),
 PosixPath('/data1/hbahk/kspec/kspecdr/resources/isoplane_commissioning/bias_00002.fits'),
 PosixPath('/data1/hbahk/kspec/kspecdr/resources/isoplane_commissioning/bias_00003.fits'),
 PosixPath('/data1/hbahk/kspec/kspecdr/resources/isoplane_commissioning/bias_00004.fits'),
 PosixPath('/data1/hbahk/kspec/kspecdr/resources/isoplane_commissioning/bias_00005.fits'),
 PosixPath('/data1/hbahk/kspec/kspecdr/resources/isoplane_commissioning/bias_00006.fits'),
 PosixPath('/data1/hbahk/kspec/kspecdr/resources/isoplane_commissioning/bias_00007.fits'),
 PosixPath('/data1/hbahk/kspec/kspecdr/resources/isoplane_commissioning/bias_00008.fits'),
 PosixPath('/data1/hbahk/kspec/kspecdr/resources/isoplane_commissioning/bias_00009.fits'),
 PosixPath('/data1/hbahk/kspec/kspecdr/resources/isoplane_commissioning/bias_00010.fits'),
 PosixPath('/data1/hbahk/kspec/kspecdr/resources/isoplane_commissioning/bias_00011.fits'),

In [ ]:
hdul = fits.open(TESTDIR / "flat_00001.fits")
hdr = hdul[0].header
# cleaned_hdr = sanitize_header_drop_unparsable(hdr)
new_hdr = convert_isoplane_header(hdr, ndfclass="MFFFF")

# add fiber table
add_fiber_table(hdul, n_fibers=16)

# just use the first frame for now
hdul[0].data = hdul[0].data[0]
# make new fits file with new header and fiber table
new_hdr["NAXIS"] = 2
new_hdr.remove("NAXIS3")
hdul[0].header = new_hdr

new_fpath = TESTDIR / "flat_00001c.fits"
hdul.writeto(new_fpath, overwrite=True)

In [ ]:
hdul.info()

In [ ]:
Table(hdul[1].data)

In [ ]:
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111)
ax.imshow(hdul[0].data, cmap="gray")
plt.show()

In [ ]:
args = {"IMAGE_FILENAME": new_fpath.as_posix()}

im = ImageFile(new_fpath.as_posix(), mode='READ')
im.open()


make_im(new_fpath.as_posix(),
        cosmic_ray_method='NONE',
        verbose=False)

In [ ]:
fpath_im = new_fpath.parent / (new_fpath.stem + "_im.fits")
args = {"IMAGE_FILENAME": fpath_im.as_posix()}

im = ImageFile(fpath_im.as_posix(), mode='READ')
im.open()
img_data, var_data, fibre_types = read_instrument_data(im, 6)

In [ ]:
plt.imshow(img_data, cmap='gray', origin='lower')
plt.show()